<a href="https://colab.research.google.com/github/Parvathy-Jayakumaran/Stackoverflow-clone/blob/main/DESEQ2_rG4_prediction_TRUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading all the libraries in advance for keeping the codes and the libraries separate

Uploaded the files of interest, for reading and analysing the data

The codeline where the DEGs are filtered. Using the same pvalue and log foldchange parameter like before

Use the rG4detector for rG4 prediction from the list of FASTA files

Running the R libraries and total Code in here to compare together

In [ ]:
install.packages("reticulate")
library(reticulate)


In [ ]:
install.packages("BiocManager")
install.packages("ggrepel")
install.packages("readxl")
install.packages("tidyverse")
install.packages("dplyr")

BiocManager::install("pqsfinder")
BiocManager::install("biomaRt")
BiocManager::install("Biostrings")

In [ ]:
install.packages("readxl")

In [ ]:
install.packages("ggrepel")
if (!requireNamespace("DESeq2", quietly = TRUE)) {
  BiocManager::install("DESeq2")
}

BiocManager::install("clusterProfile")
BiocManager::install("gogadget")

The entire R code for the original file

In [ ]:
library(dplyr)
library(tidyverse)
library(readxl)
library(DESeq2)
library(Biostrings)
library(biomaRt)
library(pqsfinder)
library(ggrepel)


sheet1 <- read_excel(
  "/content/Wild Untreated Total_vs_Mutant Untreated Total2.Differential_Expression.xlsx",
  sheet = 1
)
sheet2 <- read_excel(
  "/content/Wild Untreated Total_vs_Mutant Untreated Total2.Differential_Expression.xlsx",
  sheet = 2
)
sheet3 <- read_excel(
  "/content/Wild Untreated Total_vs_Mutant Untreated Total2.Differential_Expression.xlsx",
  sheet = 3
)
degs <- subset(sheet1, pvalue < 0.05 & abs(log2FoldChange) < 1)
head(degs[order(degs$padj), 1])

degs <- subset(sheet1, pvalue < 0.05 & abs(log2FoldChange) < 1) #The pvalue at 0.05 gave only 1 and at 0.001 gavenone
head(degs[order(degs$padj), 1])
nrow(degs)

write.table(
  degs,
  file = "degs_HU_Polysome.tsv",
  quote = FALSE,
  sep = "\t",
  row.names = FALSE,
  col.names = TRUE # Typically better to keep column names in output file
)
gene_ids <- degs$Id
gene_ids <- gene_ids[gene_ids != ""]
gene_ids <- na.omit(gene_ids)
gene_ids <- unique(gene_ids)
library(biomaRt)


mart <- useMart(biomart = "ensembl", dataset = "scerevisiae_gene_ensembl")

# Attributes to retrieve: ensembl_gene_id, ensembl_transcript_id, and cdna sequence
attributes_feature <- c(
  "ensembl_gene_id",
  "ensembl_transcript_id",
  "cdna",
  "sgd_gene"
)

attributes_structure <- c(
  "ensembl_gene_id",
  "5_utr_start",
  "5_utr_end",
  "3_utr_start",
  "3_utr_end",
  "description"
)


# Filter to use - for example 'sgd_gene' as the IDs are SGD locus tags
filter <- "sgd_gene"

# Clean gene_ids vector before query
gene_ids_clean <- unique(na.omit(gene_ids))
gene_ids_clean <- gene_ids_clean[gene_ids_clean != ""]

# Query sequences with getBM in batches to avoid overload
chunk_size <- 50
results_list <- list()

results_list1 <- list()

for (i in seq(1, length(gene_ids_clean), by = chunk_size)) {
  chunk <- gene_ids_clean[i:min(i + chunk_size - 1, length(gene_ids_clean))]

  res1 <- getBM(
    attributes = attributes_structure,
    filters = filter,
    values = chunk,
    mart = mart
  )
  results_list1[[length(results_list1) + 1]] <- res1
}

seq_data <- do.call(rbind, results_list1)
print(nrow(degs))
head(seq_data)
head(gene_ids)
head(seq_data)
seq_data <- getSequence(
  id = gene_ids,
  type = "ensembl_gene_id",
  seqType = "cdna",
  mart = mart
)
head(seq_data)

rna_seqs <- chartr("T", "U", seq_data$cdna)
rna_seqs <- RNAStringSet(rna_seqs)
head(rna_seqs)

# Downloading the sequences as FASTA
writeXStringSet(
  rna_seqs,
  filepath = "rna_sequences_HU_Polysome.fasta",
  format = "fasta"
)

library(pqsfinder)
results <- lapply(rna_seqs, pqsfinder)
num_rG4s <- sapply(results, length)
head(results)
hist(
  num_rG4s,
  main = "Number of rG4 motifs per transcripts",
  xlab = "Count of rG4 motifs"
)
names(rna_seqs) <- seq_data$ensembl_gene_id
names(results) <- names(rna_seqs)
head(names(results))

rG4_positive_transcripts <- names(results)[sapply(results, length) > 0]
print(rG4_positive_transcripts)
write.table(
  rG4_positive_transcripts,
  file = "rG4_positive_transcripts_HU_Polysome.tsv",
  quote = FALSE,
  sep = "\t",
  row.names = FALSE,
  col.names = FALSE
)

get_motif_details <- function(pq_obj) {
  if (length(pq_obj) == 0) {
    return(NULL)
  }
  data.frame(
    start = start(pq_obj),
    end = end(pq_obj),
    score = mcols(pq_obj)$score,
    sequence = as.character(pq_obj),
    stringsAsFactors = FALSE
  )
}
motif_details_list <- mapply(
  function(pq_obj, ensemble_id) {
    df <- get_motif_details(pq_obj)
    if (!is.null(df)) {
      df$transcript_id <- ensemble_id
    }
    df
  },
  results[names(results) != ""],
  names(results[names(results) != ""]),
  SIMPLIFY = FALSE
)

motif_details <- bind_rows(motif_details_list)
head(motif_details)

write.table(
  motif_details,
  file = "rG4_motif_details_list_HU_Polysome.tsv",
  quote = FALSE,
  sep = "\t",
  row.names = FALSE,
  col.names = TRUE # Typically better to keep column names in output file
)
#Plotting the curves - Volcano Plot
degs$color_group <- ifelse(
  degs$Id %in% rG4_positive_transcripts,
  "rG4_positive",
  "Other"
)
library(ggplot2)
library(ggrepel)

ggplot(degs, aes(x = log2FoldChange, y = -log10(pvalue), color = color_group)) +
  geom_point(alpha = 0.7) +
  scale_color_manual(values = c("rG4_positive" = "red", "Other" = "grey")) +
  geom_text_repel(
    data = subset(degs, color_group == "rG4_positive"),
    aes(label = Id),
    size = 3,
    box.padding = 0.3,
    point.padding = 0.5,
    segment.color = "grey50"
  ) +
  theme_minimal() +
  labs(
    title = "Volcano Plot: rG4 Positive vs Other Genes",
    x = "log2 Fold Change",
    y = "-log10 p-value"
  )

#MA Plot: Shows fold change vs Mean expression
library(ggplot2)
ggplot(degs, aes(x = baseMean, y = log2FoldChange, color = color_group)) +
  geom_point(alpha = 0.7) +
  scale_color_manual(values = c("rG4_positive" = "red", "Other" = "grey")) +
  geom_text_repel(
    data = subset(degs, color_group == "rG4_positive"),
    aes(label = Id),
    size = 3
  ) +
  labs(
    title = "MA Plot: Highlighting rG4 Positive Genes",
    x = "Mean Expression (baseMean)",
    y = "log2 Fold Change"
  ) +
  theme_minimal()


Testing the rG4 predictors in rG4detector

In [ ]:
names(rna_seqs)
head(rna_seqs)
print(head(names(rna_seqs)))
writeXStringSet(
  rna_seqs,
  filepath = "rna_sequences_HU_Polysome.fasta",
  format = "fasta"
)


# Read first 10 lines of the FASTA file
lines <- readLines("rna_sequences_HU_Polysome.fasta", n = 10)
print(lines)


In [ ]:
# 1. Load required libraries (install if necessary)
if (!requireNamespace("BiocManager", quietly=TRUE)) install.packages("BiocManager")
BiocManager::install(c("clusterProfiler", "org.Hs.eg.db", "enrichplot", "pathview", "GO.db"))

library(clusterProfiler)
library(org.Hs.eg.db)  # Change for your organism, e.g. org.Mm.eg.db for mouse
library(enrichplot)
library(pathview)
library(GO.db)

# 2. Load your DEG results (example: CSV with columns gene_id, pvalue, padj, log2FC)

# 3. Filter significant DEGs (adjust thresholds as needed)
sig_degs <- subset(degs, padj < 0.05 & abs(log2FC) > 1)
gene_ids <- sig_degs$Id  # e.g. Ensembl IDs or gene symbols

# 4. Convert gene IDs to ENTREZ for KEGG (change fromType as per your IDs)
entrez_ids <- bitr(gene_ids, fromType="ENSEMBL", toType="ENTREZID", OrgDb=org.Hs.eg.db)
entrez_list <- unique(entrez_ids$ENTREZID)

# 5. GO Enrichment Analysis (biological process, molecular function, cellular component)
ego <- enrichGO(
  gene = gene_ids,
  OrgDb = org.Hs.eg.db,
  keyType = "ENSEMBL",   # Adjust if you have SYMBOL or ENTREZ
  ont = "ALL",
  pAdjustMethod = "BH",
  pvalueCutoff = 0.05,
  qvalueCutoff = 0.2,
  readable = TRUE
)

# 6. Visualize GO results
dotplot(ego, showCategory=15, title="GO enrichment analysis")
barplot(ego, showCategory=15)

# 7. KEGG Pathway Enrichment Analysis
ekegg <- enrichKEGG(
  gene = entrez_list,
  organism = "hsa",   # "mmu" for mouse, etc.
  pvalueCutoff = 0.05
)

# 8. Visualize KEGG results
dotplot(ekegg, showCategory=15, title="KEGG Pathway Enrichment")

# 9. Optional: Visualize a specific KEGG pathway with fold change overlay
gene_fc <- sig_degs$log2FC
names(gene_fc) <- entrez_ids$ENTREZID[match(sig_degs$gene_id, entrez_ids$ENSEMBL)]

# Example KEGG pathway ID: hsa04110 (Cell cycle)
pathview(gene.data = gene_fc, pathway.id = "hsa04110", species = "hsa")

# 10. Export results to CSV files
write.csv(as.data.frame(ego), "GO_enrichment_results.csv", row.names=FALSE)
write.csv(as.data.frame(ekegg), "KEGG_enrichment_results.csv", row.names=FALSE)


PUtting rG4 detector into this

In [ ]:
file.exists("/content/Mydrive/rna_sequences_HU_Polysome.fasta")
file.exists("/content/Mydrive/rG4detector/model")


In [ ]:
install.packages(c("shiny", "G4Hunter", "other-required-packages"))


In [ ]:
library(shiny)

In [ ]:
system("git clone https://github.com/LacroixLaurent/G4Hunter")
system("git clone https://github.com/LacroixLaurent/G4HunterSeeker")

list.files("/content/G4HunterSeeker")



In [ ]:
library(shiny)
runApp("/content/G4HunterSeeker")
